In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image
from glob import glob


In [2]:
'''
Parametrização
'''

asset = 'VALE3'
time_frame = '1d'
file_name = f'{asset}_{time_frame}.xlsx'
data_path = Path(f'./collected_data/{file_name}')
charts_path = Path(f'./generated_data/{asset}')


In [3]:
'''
Importa os dados do ativo e renomeia as colunas
'''

inplace = True
df_input = pd.read_excel(data_path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'})


In [4]:
df_input.head()

,Open,High,Low,Close,Volume,MME009,MMA021,MMA200,MMA051,MMA080,MMA400,MME400,Média Móvel W [400]
Data,,,,,,,,,,,,,
1996-05-02,1.23,1.24,1.23,1.23,540980.0,1.24,1.27,1.37,1.35,1.34,1.27,1.27,1.27
1996-05-03,1.24,1.24,1.23,1.23,93125.0,1.24,1.27,1.36,1.35,1.34,1.27,1.27,1.27
1996-05-06,1.23,1.23,1.23,1.23,53050.0,1.24,1.26,1.36,1.35,1.34,1.28,1.27,1.27
1996-05-08,1.23,1.23,1.23,1.23,310250.0,1.23,1.25,1.36,1.34,1.34,1.28,1.27,1.27
1996-05-14,1.23,1.23,1.22,1.22,168990.0,1.23,1.25,1.36,1.34,1.34,1.28,1.27,1.27


In [5]:
df_input.tail()

,Open,High,Low,Close,Volume,MME009,MMA021,MMA200,MMA051,MMA080,MMA400,MME400,Média Móvel W [400]
Data,,,,,,,,,,,,,
2020-08-20,61.82,63.50,61.67,62.95,1.611985e+09,62.02,61.23,51.31,58.63,55.50,50.03,50.34,45.98
2020-08-21,62.41,62.79,61.51,62.20,1.688418e+09,62.05,61.39,51.37,58.79,55.69,50.06,50.40,46.02
2020-08-24,62.94,63.13,62.24,62.96,1.116583e+09,62.24,61.60,51.45,58.98,55.92,50.09,50.46,46.06
2020-08-25,62.80,62.84,61.05,61.62,1.439559e+09,62.11,61.61,51.52,59.13,56.14,50.11,50.52,46.10
2020-08-26,61.66,62.35,61.24,61.75,1.470004e+09,62.04,61.68,51.58,59.26,56.36,50.13,50.57,46.14


In [6]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)

# df.head()


In [7]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")
    

Data Frame válido.


In [8]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data

IMPLEMENTAR MULTIPROCESSAMENTO PARA GILIZAR A CRIAÇÃO DAS IMAGENS
>>> import multiprocessing
'''

def date_delta(start_date, delta): # Função sem uso por enquanto
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = delta)
    return end_date.strftime('%Y-%m-%d')


def gen_chart(df, date, trade_delta, min_profit, chart_delta, add, save_name):
    end_index = df.index.get_loc(date)
    start_index = end_index - chart_delta
    savefig = './generated_data/plot.png'
    
    start_trade = df['Open'][end_index + 1]
    end_trade = df['Close'][end_index + trade_delta]
    if end_trade/start_trade >= 1 + min_profit/100:
        trade_class = 'long'
    elif end_trade/start_trade <= 1 - min_profit/100:
        trade_class = 'short'
    else:
        trade_class = 'wait'

    df_plot = df.iloc[start_index:end_index,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g',down = 'r',
                               wick = 'inherit',
                               edge = 'inherit',
                               volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc,
                           facecolor = erase_color,
                           edgecolor = erase_color,
                           figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    plot = mpf.plot(df_plot, type = 'candle',
                             volume = True,
                             style = s,
                             addplot = addplot,
                             savefig = savefig)

    uncropped_plot = Image.open(savefig)
    box = (uncropped_plot.size[0]*0.202,
           uncropped_plot.size[1]*0.11,
           uncropped_plot.size[0]*0.88,
           uncropped_plot.size[1]*0.85)

    cropped_plot = uncropped_plot.crop(box).save(f'{save_name}_{trade_class}.png')

    return (start_trade, end_trade, trade_class)
    

In [9]:
'''
Chamamento das funções de geração dos gráficos
Implementar o loop for
'''

charts_path.mkdir(parents=True, exist_ok=True)
date_range = pd.date_range(start='2014-06-28', end='2020-07-30')
trade_delta = 2
min_profit = 1 # in %
chart_delta = 60
ind_columns = list(df.columns)[5:10]

count = 0
for date in date_range:
    try:
        file_date = date.strftime('%Y-%m-%d')
        save_name = charts_path/f'{asset}_{file_date}'
        gen_chart(df, date, trade_delta, min_profit, chart_delta, ind_columns, save_name)
        count += 1
    except KeyError or IndexError:
        continue
print(f'{count} imagens geradas.')

1507 imagens geradas.
